# 데이터 처리

- 아래는 model을 2개의 문장을 fine-tuning하는 과정

In [1]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# GLUE datasets을 이용해보자

In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

- train 데이터만 변수에 저장

In [4]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

- 데이터셋의 특성을 보기

In [5]:
raw_datasets['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

-  토크나이저

In [6]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

# 데이터 전처리

- 아래 task는 두문장의 관계를 모델링하는것

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

inputs = tokenizer("This is the first sentence.", "This is the second one.", "hello my friend.", return_tensors="pt")
inputs


{'input_ids': tensor([[ 101, 2023, 2003, 1996, 2034, 6251, 1012,  102, 2023, 2003, 1996, 2117,
         2028, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ 101, 7592, 2026, 2767, 1012,  102]])}

In [8]:
inputs = tokenizer("This is the first sentence.", "This is the second one." )
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
tokenized_dataset = tokenizer(
    raw_datasets[ "train" ][ "sentence1" ],
    raw_datasets[ "train" ][ "sentence2" ],
    padding= True ,
    truncation= True ,
)

In [10]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [11]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [12]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
# 주어진 데이터셋에서 "train" 부분의 처음 8개 샘플을 가져옵니다.
samples = tokenized_datasets["train"][:8]
print(samples)
# "idx", "sentence1", "sentence2" 키를 제외한 모든 키와 해당 값을 딕셔너리로 필터링합니다.
filtered_samples = {}
for key, value in samples.items():
    if key not in ["idx", "sentence1", "sentence2"]:
        filtered_samples[key] = value

# 각 샘플의 "input_ids"에 대한 토큰 길이를 계산합니다.
token_lengths = [len(x) for x in filtered_samples["input_ids"]]

# 토큰 길이를 출력합니다.
print(token_lengths)


{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .', 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .', 'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .', 'Revenue in the first quarter of the year dropped 15 percent from the same period a year earlier .', 'The Nasdaq had a weekly gain of 17.27 , or 1.2 percent , closing at 1,520.15 on Friday .', 'The DVD-CCA then appealed to the state Supreme Court .'], 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold 

In [15]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [16]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

# Trainer API
- 데이터세트에서 쉽게 fine-tune 혹은 쉽게 학습 할수 있는 API
- 모델 및 학습 하이퍼 파라미터를 사용하여 모든 종류의 설정(CPU, GPU, TPU)에서 학습을 수행하고 평가할수 있음

1. 모델의 모든 하이퍼  파라미터를 포함하는 클래스를 Trainer에 정의하기

In [17]:
! pip install -U accelerate
! pip install -U transformers

from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

2. 모델을 chekpoint에서 불러와서 정의하기

- 이는 BERT가 문장 쌍을 분류하는 데 사전 학습되지 않았기 때문에 사전 학습된 모델의 헤드를 삭제하고 대신 시퀀스 분류에 적합한 새로운 헤드를 추가했기 때문입니다. 경고는 일부 가중치(삭제된 사전 훈련 헤드에 해당하는 가중치)가 사용되지 않았으며 일부 가중치(새 헤드에 대한 가중치)가 무작위로 초기화되었음을 나타냅니다.
- Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

In [18]:
from transformers import AutoModelForSequenceClassification
checkpoint = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3. Trainer를 이용하여 사전 훈련된 모델과 나의 데이터셋을 이용하여 학습을 진행한다

In [20]:
# transformers 라이브러리에서 Trainer 클래스를 가져옵니다.
from transformers import Trainer

# Trainer 인스턴스를 생성합니다. 다음 매개변수를 전달합니다.
trainer = Trainer(
    # model: fine-tuning할 사전 훈련된 모델을 전달합니다.
    model,
    # training_args: 훈련 구성 매개변수를 전달합니다.
    training_args,
    # train_dataset: 토큰화된 훈련 데이터셋을 전달합니다.
    train_dataset=tokenized_datasets["train"],
    # eval_dataset: 토큰화된 검증 데이터셋을 전달합니다.
    eval_dataset=tokenized_datasets["validation"],
    # data_collator: 배치 처리 및 패딩을 위한 데이터 콜레이터를 전달합니다.
    data_collator=data_collator,
    # tokenizer: 토크나이저를 전달합니다.
    tokenizer=tokenizer,
)
trainer.train()

Step,Training Loss
500,0.529500
1000,0.270500


TrainOutput(global_step=1377, training_loss=0.3236880582921645, metrics={'train_runtime': 161.0728, 'train_samples_per_second': 68.317, 'train_steps_per_second': 8.549, 'total_flos': 405626802939840.0, 'train_loss': 0.3236880582921645, 'epoch': 3.0})

4. 평가

In [21]:
# 훈련된 모델을 사용하여 검증 데이터셋에 대한 예측을 수행합니다.
predictions = trainer.predict(tokenized_datasets["validation"])

# 예측 결과의 형태(shape)와 레이블의 형태(shape)를 출력합니다.
print(predictions.predictions.shape, predictions.label_ids.shape)


(408, 2) (408,)


In [31]:
import numpy as np
# 예측값을 argmax로 변경 정답지와 비교할 수 있도록
preds = np.argmax(predictions.predictions, axis=-1)
print(preds)

[1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0
 0 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 0 1 1 1
 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0
 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0
 1 0 1 1 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1
 0 1 1 1 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0
 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 0
 1]


In [30]:
# 평가 함수 라이브러리 가져오기
import evaluate

# MRPC 평가 지표 로드
metric = evaluate.load("glue", "mrpc")  # GLUE 벤치마크의 MRPC 태스크를 위한 평가 지표 로드

# 예측 결과와 실제 정답을 바탕으로 평가 수행
metric.compute(predictions=preds, references=predictions.label_ids)  # 예측 결과와 정답 비교

{'accuracy': 0.8382352941176471, 'f1': 0.888135593220339}

In [32]:
def compute_metrics(eval_preds):  # 평가 지표 계산 함수
    # MRPC 평가 지표 로드
    metric = evaluate.load("glue", "mrpc")

    # logits과 레이블 분리
    print(eval_preds)

    # logits에서 가장 높은 확률의 레이블을 예측값으로 추출
    predictions = np.argmax(logits, axis=-1)

    # 예측값과 실제 레이블을 기반으로 평가 수행 및 결과 반환
    return metric.compute(predictions=predictions, references=labels)

In [38]:
# 훈련 인자 설정
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")  # 훈련 인자 생성

# 모델 생성
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)  # 사전 훈련된 모델 로드

# 훈련기 생성
trainer = Trainer(  # 훈련기 생성
    model,               # 훈련할 모델
    training_args,       # 훈련 인자
    train_dataset=tokenized_datasets["train"],  # 훈련 데이터셋
    eval_dataset=tokenized_datasets["validation"],  # 검증 데이터셋
    data_collator=data_collator,  # 데이터 전처리 함수
    tokenizer=tokenizer,   # 토크나이저
    compute_metrics=compute_metrics,  # 평가 함수
)

# 훈련 시작
trainer.train()  # 훈련 프로세스 시작

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.417104,0.816176,0.876033
2,0.523200,0.482961,0.843137,0.888502
3,0.300300,0.751781,0.835784,0.883882


TrainOutput(global_step=1377, training_loss=0.3328878642340544, metrics={'train_runtime': 106.3173, 'train_samples_per_second': 103.502, 'train_steps_per_second': 12.952, 'total_flos': 405626802939840.0, 'train_loss': 0.3328878642340544, 'epoch': 3.0})